<a href="https://colab.research.google.com/github/manuelespinosa/MEREE_DataAnalysis/blob/main/MetodologiaDeLaInvestigacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from time import sleep
from tqdm import tqdm

# API Key de AEMET OpenData (necesitas registrarte en opendata.aemet.es para obtenerla)
api_key = "INSERTA_AQUI_API_KEY_DE_AEMET"

# URL base de la API
base_url = "https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/{}/fechafin/{}/estacion/{}"

# Fechas de inicio y fin (formato: YYYY-MM-DDTHH:MM:SSUTC)
fecha_ini = "1945-01-01T00:00:00UTC"
fecha_fin = "2025-02-28T23:59:00UTC"
estacion = "6001"  # ID de Tarifa


# Función para generar tramos de 6 meses
def generar_tramos(fecha_ini, fecha_fin):
    tramos = []
    inicio = datetime.strptime(fecha_ini, "%Y-%m-%dT%H:%M:%SUTC")
    fin = datetime.strptime(fecha_fin, "%Y-%m-%dT%H:%M:%SUTC")
    while inicio < fin:
        fin_tramo = inicio + timedelta(days=180)  # Aproximadamente 6 meses
        if fin_tramo > fin:
            fin_tramo = fin
        tramos.append((inicio.strftime("%Y-%m-%dT%H:%M:%SUTC"), fin_tramo.strftime("%Y-%m-%dT%H:%M:%SUTC")))
        inicio = fin_tramo + timedelta(days=1)
    return tramos


In [ ]:
tramos = generar_tramos(fecha_ini, fecha_fin)

# Recolectar datos con reintentos y barra de progreso
datos_totales = []
headers = {"api_key": api_key}
max_intentos = 5  # Número máximo de reintentos por tramo

# Usar tqdm para mostrar barra de progreso
for ini, fin in tqdm(tramos, desc="Recolectando datos"):
    intentos = 0
    exito = False
    while intentos < max_intentos and not exito:
        try:
            url = base_url.format(ini, fin, estacion)
            response = requests.get(url, headers=headers, timeout=30)
            response.raise_for_status()
            datos_url = response.json()["datos"]
            datos_response = requests.get(datos_url, timeout=30)
            datos_response.raise_for_status()
            datos = datos_response.json()
            datos_totales.extend(datos)
            exito = True
        except (requests.exceptions.RequestException, ValueError) as e:
            intentos += 1
            if intentos == max_intentos:
                print(f"Error definitivo en tramo {ini} - {fin} tras {max_intentos} intentos: {e}")
            else:
                print(f"Error en tramo {ini} - {fin}, intento {intentos}/{max_intentos}: {e}")
                sleep(10 * intentos)
    sleep(5)  # Pausa mínima entre tramos

# Convertir a DataFrame
df = pd.DataFrame(datos_totales)

# Procesar datos
df["tmed"] = pd.to_numeric(df["tmed"].str.replace(",", "."), errors="coerce")
df["prec"] = pd.to_numeric(df["prec"].str.replace(",", "."), errors="coerce")
df["fecha"] = pd.to_datetime(df["fecha"])
df = df.dropna(subset=["tmed", "prec"])
df["año"] = df["fecha"].dt.year
anual = df.groupby("año").agg({"tmed": "mean", "prec": "sum"}).reset_index()

print("Datos procesados con éxito. Filas:", len(df))

Recolectando datos:  23%|██▎       | 38/162 [04:12<13:42,  6.63s/it]


KeyboardInterrupt: 

In [ ]:
df.to_csv("datosTarifa.csv")

,fecha,indicativo,nombre,provincia,altitud,tmed,prec,tmin,horatmin,tmax,...,presMax,horaPresMax,presMin,horaPresMin,hrMedia,hrMax,horaHrMax,hrMin,horaHrMin,año
0,2020-01-01,6001,TARIFA,CADIZ,32,14.1,0.0,"12,4",23:40,"15,8",...,"1027,5",11,"1024,2",05,76,85,23:10,69,Varias,2020
1,2020-01-02,6001,TARIFA,CADIZ,32,14.0,0.0,"12,2",02:20,"15,7",...,"1027,7",11,"1025,0",04,78,87,Varias,69,Varias,2020
2,2020-01-03,6001,TARIFA,CADIZ,32,12.4,0.0,"9,1",08:00,"15,8",...,"1028,7",10,"1025,9",15,79,95,08:00,62,15:40,2020
3,2020-01-04,6001,TARIFA,CADIZ,32,10.9,0.0,"6,5",08:20,"15,3",...,"1027,5",10,"1024,2",15,79,92,08:20,66,13:40,2020
4,2020-01-05,6001,TARIFA,CADIZ,32,14.6,0.0,"13,8",01:50,"15,4",...,"1024,4",00,"1021,1",Varias,76,85,02:10,68,22:50,2020
